# Using IPython Notebooks

IPython notebooks are made up of a number of 'cells'. Each cell can either run some python code or contain text.

To run a cell, click on it or move to it with up and down arrows and press 'Shift + Enter'. Alternatively, you can press the 'play' button in the toolbar. Any output from the cell (text or plots) will apprear below the cell after you run it Try running the cell below!

In [1]:
print "Hello World!"

Hello World!


Before you can do much in Python, you'll probably need to import one or a few libraries. NumPy is a matrix library that is used in scientific python applications. The 'import X as Y' statement loads a library X under the alias Y.

In [2]:
import numpy as np

IPython can tell you about the properties of functions and modules/libraries. One way to get this is to 'tab complete'. Given a library, 'np', you can query the function in the library by typing 'np.' and pressing 'Tab'.

In [3]:
np.

SyntaxError: invalid syntax (<ipython-input-3-a58a88215da2>, line 1)

To learn about a function, you can type the functions name then a question mark and run the cell.

In [4]:
np.zeros?

# Positive Only Sparse Coding on MNIST

This code implements Positive Only Sparse Coding on MNIST.

Lets break that down.

Sparse Coding is an algorithm that seeks to disentangle the underlying generative factors of data. In this particular case, we are considering the dataset, MNIST, which is a collection of digits. Intuitively speaking, the underlying generative factors of digits are pen strokes. When these strokes are combined in the right way, we get digits. We will see that sparse coding discovers this underlying structure. 

In sparse coding, we define the following:
$I$ is an image, $D$ is a dictionary of underlying generative factors, and $A$ the sparse coefficients. In long form, 

$$I(x) = d_1(x) * a_1 + d_2(x) * a_2 + \ldots$$

where the dictionary elements are the same for any image in the data set, and the coefficients, $a$, are different for each image. 

In order to learn the dictionary elements $D$, and the sparse coefficients, $A$, we minimize the following objective function:

$$E = |I - A * D| ^ 2 + |A|_1 \qquad \sum_x d_i(x) = 1$$

(Note $D$ is number of dictionary elements by the number of pixels). To minimize this function, we do the following steps:

0. Choose an initial value of $D$.

1. Choose a batch of images $I$.

2. Minimize $E$ with respect to $A$ - we use FISTA which is a slighly better version of gradient descent.

3. Keeping that value of $A$, reduce $E$ by one gradient step in the direction of $\nabla_D E$. 

4. Return to step 1. 

### Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import savemat, loadmat
from scipy.stats import probplot, expon

from utils.rf_plot import show_fields

from network import Network

### Load Data

In [2]:
data_dir = "data/"
data_fil = "mnist.mat"
output_dir = 'output/'

data = loadmat(data_dir + data_fil)
IMAGES = data['IMAGES']
LABELS = data['LABELS']

# Set basic parameters
(K, L_img, L_img) = IMAGES.shape
print('Number of images: '+str(K))
K # Number of base images
L_img # Linear size of images from the data
N_pix_img = L_img ** 2

# Scale Images to have standard deviation one
IMAGES = IMAGES / np.std(IMAGES.astype(float), axis = (1, 2), keepdims = True)
data = IMAGES.reshape(60000, 14 * 14) # FIXME

pos_only = True # Positive Only Sparse coding if True
N_sp = 81 # Number of sparse dictionary elements
lamb = 0.01 # coefficient for the sparse coefficients
eta = 0.01 # Dictionary Learning Step Size

Number of images: 60000


### Create Sparse Coding Network

In [3]:
net = Network(N_sp, N_pix_img, lamb, eta)

In [5]:
net.infer_A(data[0:100], n_g_steps=150, track_cost=True)

127.462770732
79.777919671
79.7066654721
79.6997656647
79.698510812
79.6982785892
79.6982396807
79.6982301096
79.6982239844
79.6982201046


array([[ -2.31969396e+00,  -1.00446510e-01,  -3.36523322e-01, ...,
         -7.49618053e-01,   2.42939274e+00,  -1.06487341e+00],
       [ -1.95277891e+00,  -1.19886899e+00,  -1.12818603e+00, ...,
         -5.82655240e-01,   3.57340892e+00,  -7.97927878e-01],
       [ -1.85314216e+00,  -3.94823691e-01,   1.19846855e+00, ...,
         -1.53605406e+00,   3.25516039e-01,   2.12325818e-03],
       ..., 
       [ -1.18999544e+00,  -1.23165292e+00,   4.18561189e-01, ...,
         -3.27891162e-01,   1.80302949e+00,  -9.38192452e-01],
       [ -2.19576683e+00,  -0.00000000e+00,  -3.35814004e-01, ...,
         -1.81545693e+00,   1.70621511e+00,   6.72662056e-01],
       [ -8.32864139e-01,  -3.11845410e-01,  -1.11915023e+00, ...,
         -5.51647989e-01,   3.91625797e+00,  -6.46811169e-01]])

Inspect the functions in the network.

In [ ]:
net.

In [ ]:
cost_list = []

In [ ]:
I_idx = train(100, Alpha, 0.2, cost_list, show_costs=False)

In [ ]:
I_idx = train(100, Alpha, 0.1, cost_list, show_costs=False)

In [ ]:
I_idx = train(100, Alpha, 0.03, cost_list, show_costs=False)

In [ ]:
I_idx = train(1, Alpha, 0.01, cost_list, show_costs=False, N_g_itr = 300)

In [ ]:
plt.plot(np.log(np.array(cost_list)))
plt.xlabel('Iteration')
plt.ylabel('Log(cost)')

In [ ]:
plt.figure(figsize=(12,10))
show_fields(t_D.get_value(), cmap = plt.cm.gray, pos_only = True)
#plt.savefig(output_dir + 'mnist_basis.png', dpi = 200)

In [ ]:
plt.figure(figsize=(12, 10))
q = np.random.randint(N_bat)
plt.subplot(2, 2, 1)
plt.title('Reconstruction')
plt.imshow(np.dot(t_A.get_value(), t_D.get_value())[q].reshape(L_pat, L_pat),
           interpolation = 'nearest',
           cmap = plt.cm.gray, vmin = 0, vmax = t_I.get_value()[I_idx][q].reshape(L_pat, L_pat).max())
plt.colorbar()
plt.subplot(2, 2, 2)
plt.title('Orignal Image')
plt.imshow(t_I.get_value()[I_idx][q].reshape(L_pat, L_pat),
           interpolation = 'nearest',
           cmap = plt.cm.gray)
plt.colorbar()
plt.subplot(2, 2, 3)

plt.hist(t_A.get_value()[q])

sort_idx = np.argsort(t_A.get_value()[q])[::-1]
N_active = np.sum(t_A.get_value()[q] > 0.0)
active_idx = sort_idx[0:N_active]

plt.title('Histogram of sparse Coefficients: \n Number of active coefficients %d' % N_active)
plt.xlabel('Coefficient Activity')

plt.subplot(2, 2, 4)
show_fields(t_D.get_value()[active_idx] * 
            t_A.get_value()[q][active_idx][:, np.newaxis], 
            cmap = plt.cm.gray, pos_only = True)
plt.title('Active Dictionary Elements \n Scaled by their activations')

In [ ]:
As = t_A.get_value().ravel()
probplot(As[As > 0], dist = expon, plot = plt)

Details on FISTA:

Since the objective function has an absolute value, typical gradient descent approaches converge slowly. Thus there are special purpose gradient descent methods that minimize functions that are in the form $$f(x) + g(x)$$ where $f(x)$ is a continuously differentiable, convex function and $g(x)$ is a convex, but not continuously differentiable function, such as $g(x) = \alpha |x|$. One such method is called FISTA, or the Fast Iterative Shrinkage-Threshold Algorithm. 

The core kernel of the FISTA algorithm is the ISTA step:

Define
$$p_L(y) = \text{argmin}_x \, g(x) + L/2 * ||x- g(y)||^2$$ where $$g(y) = y - \frac{1}{L} \nabla f(y)$$

and where $L$ is the constant such that $$||\nabla f(x) - \nabla f(y)|| \le L ||x - y||$$

When $g(x) = \alpha|x|_1$, then $$p_L(y) = h_\theta(y)\qquad h_\theta(y) = \text{sign}(y)(|y|-\theta)\qquad \theta = \frac{\alpha}{L}$$
$h$ is applied pointwise to $y$ and is the shrinkage function. Simplying calculating $x_{t+1} = p_L(x_t)$ is the ISTA algorithm. If we more intelligently choose our new value to probe our function, then we get faster convergence. The FISTA algorithm is as follows:

1. Initialize $y_0 = x_0 = X0$, $t_0=1$. 

2. For $k \ge 0$, iterate the following:

$$x_{k+1} = p_L(y_k)\qquad t_{k+1} = 0.5 * (1 + \sqrt{1 + 4 * t_k ^2})\qquad y_{k+1} = x_{k+1} + \frac{t_k - 1}{t_{k+1}} * (x_{k+1} - x_k)$$

In [ ]:
#savemat(output_dir + 'mnist_dictionary.mat', 
#        {'D': t_D.get_value(), 'Alpha': Alpha,
#         'Algorithm': 'FISTA', 'Normalization': 'Equal Standard Deviation'})